I referenced the Andy Harless' Kernel, https://www.kaggle.com/aharless/xgboost-cv-lb-284

In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from numba import jit
import time
import gc

In [2]:
MAX_ROUNDS = 2000
OPTIMIZE_ROUNDS = True
LEARNING_RATE = 0.07
EARLY_STOPPING_ROUNDS = 50
# Note: I set EARLY_STOPPING_ROUNDS high so that (when OPTIMIZE_ROUNDS is set)
#       I will get lots of information to make my own judgment.  You should probably
#       reduce EARLY_STOPPING_ROUNDS if you want to do actual early stopping.


I recommend initially setting ```MAX_ROUNDS``` fairly high and using ```OPTIMIZE_ROUNDS``` to get an idea of the appropriate number of rounds (which, in my judgment, should be close to the maximum value of best_ntree_limit among all folds, maybe even a bit higher if your model is adequately regularized...or alternatively, you could set ```verbose=True``` and look at the details to try to find a number of rounds that works well for all folds). Then I would turn off ```OPTIMIZE_ROUNDS``` and set ```MAX_ROUNDS``` to the appropraite number of total rounds.

The problem with "early stopping" by choosing the best round for each fold is that it overfits to the validation data. It's therefore liable not to produce the optimal model for predicting test data, and if it's used to produce validation data for stacking/ensembling with other models, it would cause this one to have too much weight in the ensemble. Another possibility (and the default for XGBoost, it seems) is to use the round where the early stop actually happens (with the lag that verifies lack of improvement) rather than the best round. That solves the overfitting problem (provided the lag is long enough), but so far it doesn't seem to have helped. (I got a worse validation score with 20-round early stopping per fold than with a constant number of rounds for all folds, so the early stopping actually seemed to underfit.)

In [3]:
# Compute gini

# from CPMP's kernel https://www.kaggle.com/cpmpml/extremely-fast-gini-computation

@jit
def eval_gini(y_true, y_prob):
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    ntrue = 0
    gini = 0
    delta = 0
    n = len(y_true)
    for i in range(n-1, -1, -1):
        y_i = y_true[i]
        ntrue += y_i
        gini += y_i * delta
        delta += 1 - y_i
    gini = 1 - 2 * gini / (ntrue * (n - ntrue))
    return gini

In [4]:
# Funcitons from olivier's kernel
# https://www.kaggle.com/ogrellier/xgb-classifier-upsampling-lb-0-283

def gini_xgb(preds, dtrain):
    labels = dtrain.get_label()
    gini_score = -eval_gini(labels, preds)
    return [('gini', gini_score)]

def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))

# Revised to encode validation series
def target_encode(trn_series=None,
                 val_series=None,
                 tst_series=None,
                 target=None,
                 min_samples_leaf=1,
                 smoothing=1,
                 noise_level=0):
    """
    trn_series : training categorical feature as a pd.Series
    tst_series : test categorical feature as a pd.Series
    target : target data as a pd.Series
    min_samples_leaf (int) : minimum samples to take category average into account
    smoothing (int) : smoothing effect to balance categorical average vs prior
    """ 
    assert len(trn_series) == len(target)
    assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1)
    # compute target mean
    averages = temp.groupby(by=trn_series.name)[target.name].agg(['mean', 'count'])
    # compute smoothing
    smoothing = 1 / (1+np.exp(-(averages['count'] - min_samples_leaf) / smoothing))
    # apply average function to all target data
    prior = target.mean()
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1-smoothing) + averages['mean']*smoothing
    averages.drop(['mean', 'count'], axis=1, inplace=True)
    # apply average to trn and tst series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index
    ft_val_series = pd.merge(
        val_series.to_frame(val_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=val_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_val_series.index = val_series.index
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_tst_series.index = tst_series.index
    return add_noise(ft_trn_series, noise_level), add_noise(ft_val_series, noise_level), add_noise(ft_tst_series, noise_level)

In [5]:
train_df = pd.read_csv('data/train.csv', na_values='-1')
test_df = pd.read_csv('data/test.csv', na_values='-1')

In [6]:
# from oliver
train_features = [
    "ps_car_13",  #            : 1571.65 / shadow  609.23
	"ps_reg_03",  #            : 1408.42 / shadow  511.15
	"ps_ind_05_cat",  #        : 1387.87 / shadow   84.72
	"ps_ind_03",  #            : 1219.47 / shadow  230.55
	"ps_ind_15",  #            :  922.18 / shadow  242.00
	"ps_reg_02",  #            :  920.65 / shadow  267.50
	"ps_car_14",  #            :  798.48 / shadow  549.58
	"ps_car_12",  #            :  731.93 / shadow  293.62
	"ps_car_01_cat",  #        :  698.07 / shadow  178.72
	"ps_car_07_cat",  #        :  694.53 / shadow   36.35
	"ps_ind_17_bin",  #        :  620.77 / shadow   23.15
	"ps_car_03_cat",  #        :  611.73 / shadow   50.67
	"ps_reg_01",  #            :  598.60 / shadow  178.57
	"ps_car_15",  #            :  593.35 / shadow  226.43
	"ps_ind_01",  #            :  547.32 / shadow  154.58
	"ps_ind_16_bin",  #        :  475.37 / shadow   34.17
	"ps_ind_07_bin",  #        :  435.28 / shadow   28.92
	"ps_car_06_cat",  #        :  398.02 / shadow  212.43
	"ps_car_04_cat",  #        :  376.87 / shadow   76.98
	"ps_ind_06_bin",  #        :  370.97 / shadow   36.13
	"ps_car_09_cat",  #        :  214.12 / shadow   81.38
	"ps_car_02_cat",  #        :  203.03 / shadow   26.67
	"ps_ind_02_cat",  #        :  189.47 / shadow   65.68
	"ps_car_11",  #            :  173.28 / shadow   76.45
	"ps_car_05_cat",  #        :  172.75 / shadow   62.92
	"ps_calc_09",  #           :  169.13 / shadow  129.72
	"ps_calc_05",  #           :  148.83 / shadow  120.68
	"ps_ind_08_bin",  #        :  140.73 / shadow   27.63
	"ps_car_08_cat",  #        :  120.87 / shadow   28.82
    "ps_ind_09_bin",  #        :  113.92 / shadow   27.05
    "ps_ind_04_cat",  #        :  107.27 / shadow   37.43
	"ps_ind_18_bin",  #        :   77.42 / shadow   25.97
	"ps_ind_12_bin",  #        :   39.67 / shadow   15.52
	"ps_ind_14",  #            :   37.37 / shadow   16.65
]

# add combinations
combs = [
    ('ps_reg_01', 'ps_car_02_cat'),
    ('ps_reg_01', 'ps_car_04_cat'),
]

In [7]:
# process data
id_test = test_df['id'].values
id_train = train_df['id'].values
y = train_df['target']

start = time.time()
# f-string format (over python 3.6)
for index, (f1, f2) in enumerate(combs):
    name = f1 + '_plus_' + f2
    duration = (time.time() - start) / 60
    print(f'current feature {name:>30} {index+1:>5} in {duration}', end='\n')
    train_df[name] = train_df[f1].apply(lambda x: str(x)) + '_' + train_df[f2].apply(lambda x: str(x))
    test_df[name] = test_df[f1].apply(lambda x: str(x)) + '_' + test_df[f2].apply(lambda x: str(x))
    
    # Label encode
    label_encoder = LabelEncoder()
    label_encoder.fit(list(train_df[name].values) + list(test_df[name].values))
    train_df[name] = label_encoder.transform(list(train_df[name].values))
    test_df[name] = label_encoder.transform(list(test_df[name].values))
    
    train_features.append(name)
    
X = train_df[train_features]
test_df = test_df[train_features]

f_cats = [f for f in X.columns if '_cat' in f]

current feature   ps_reg_01_plus_ps_car_02_cat     1 in 0.0
current feature   ps_reg_01_plus_ps_car_04_cat     2 in 0.10431904395421346


In [8]:
y_valid_pred = 0*y
y_test_pred = 0

In [9]:
# Set up folds
K = 5
kf = KFold(n_splits = K, random_state = 1, shuffle = True)
np.random.seed(0)

In [10]:
# Set up classifier
xgb_model = XGBClassifier(
                        n_estimator=MAX_ROUNDS,
                        max_depth=4,
                        objective='binary:logistic',
                        learning_rate=LEARNING_RATE,
                        subsample=0.8,
                        min_child_weight=6,
                        colsample_bytree=0.8,
                        scale_pos_weight=1.6,
                        gamma=10,
                        reg_alpha=8,
                        reg_lambda=1.3
                        )

In [11]:
kf

KFold(n_splits=5, random_state=1, shuffle=True)

In [12]:
# Run CV

for i, (train_index, test_index) in enumerate(kf.split(train_df)):
    
    # Create data for this fold
    y_train, y_valid = y.iloc[train_index].copy(), y.iloc[test_index]
    X_train, X_valid = X.iloc[train_index,:].copy(), X.iloc[test_index,:].copy()
    X_test = test_df.copy()
    print( "\nFold ", i)
    
    # Enocode data
    for f in f_cats:
        X_train[f + "_avg"], X_valid[f + "_avg"], X_test[f + "_avg"] = target_encode(
                                                        trn_series=X_train[f],
                                                        val_series=X_valid[f],
                                                        tst_series=X_test[f],
                                                        target=y_train,
                                                        min_samples_leaf=200,
                                                        smoothing=10,
                                                        noise_level=0
                                                        )
    # Run model for this fold
    if OPTIMIZE_ROUNDS:
        eval_set=[(X_valid,y_valid)]
        fit_model = xgb_model.fit( X_train, y_train, 
                               eval_set=eval_set,
                               eval_metric=gini_xgb,
                               early_stopping_rounds=EARLY_STOPPING_ROUNDS,
                               verbose=True
                             )
        print( "  Best N trees = ", xgb_model.best_ntree_limit )
        print( "  Best gini = ", xgb_model.best_score )
    else:
        fit_model = xgb_model.fit( X_train, y_train )
        
    # Generate validation predictions for this fold
    pred = fit_model.predict_proba(X_valid)[:,1]
    print( "  Gini = ", eval_gini(y_valid, pred) )
    y_valid_pred.iloc[test_index] = pred
    
    # Accumulate test set predictions
    y_test_pred += fit_model.predict_proba(X_test)[:,1]
    
    del X_test, X_train, X_valid, y_train
    
y_test_pred /= K  # Average test set predictions

print( "\nGini for full training set:" )
eval_gini(y, y_valid_pred)


Fold  0
[0]	validation_0-error:0.036407	validation_0-gini:-0.178161
Multiple eval metrics have been passed: 'validation_0-gini' will be used for early stopping.

Will train until validation_0-gini hasn't improved in 50 rounds.
[1]	validation_0-error:0.036407	validation_0-gini:-0.19852
[2]	validation_0-error:0.036407	validation_0-gini:-0.200389
[3]	validation_0-error:0.036407	validation_0-gini:-0.207788
[4]	validation_0-error:0.036407	validation_0-gini:-0.212626
[5]	validation_0-error:0.036407	validation_0-gini:-0.213077
[6]	validation_0-error:0.036407	validation_0-gini:-0.218615
[7]	validation_0-error:0.036407	validation_0-gini:-0.218812
[8]	validation_0-error:0.036407	validation_0-gini:-0.21989
[9]	validation_0-error:0.036407	validation_0-gini:-0.222299
[10]	validation_0-error:0.036407	validation_0-gini:-0.222874
[11]	validation_0-error:0.036407	validation_0-gini:-0.225275
[12]	validation_0-error:0.036407	validation_0-gini:-0.227745
[13]	validation_0-error:0.036407	validation_0-gini:

[29]	validation_0-error:0.036096	validation_0-gini:-0.247435
[30]	validation_0-error:0.036096	validation_0-gini:-0.247917
[31]	validation_0-error:0.036096	validation_0-gini:-0.249458
[32]	validation_0-error:0.036096	validation_0-gini:-0.249682
[33]	validation_0-error:0.036096	validation_0-gini:-0.250452
[34]	validation_0-error:0.036096	validation_0-gini:-0.250749
[35]	validation_0-error:0.036096	validation_0-gini:-0.251603
[36]	validation_0-error:0.036096	validation_0-gini:-0.252065
[37]	validation_0-error:0.036096	validation_0-gini:-0.252632
[38]	validation_0-error:0.036096	validation_0-gini:-0.254018
[39]	validation_0-error:0.036096	validation_0-gini:-0.255752
[40]	validation_0-error:0.036096	validation_0-gini:-0.256388
[41]	validation_0-error:0.036096	validation_0-gini:-0.256439
[42]	validation_0-error:0.036096	validation_0-gini:-0.256648
[43]	validation_0-error:0.036096	validation_0-gini:-0.256976
[44]	validation_0-error:0.036096	validation_0-gini:-0.257434
[45]	validation_0-error:

[60]	validation_0-error:0.037256	validation_0-gini:-0.251941
[61]	validation_0-error:0.037264	validation_0-gini:-0.252736
[62]	validation_0-error:0.037256	validation_0-gini:-0.252745
[63]	validation_0-error:0.037264	validation_0-gini:-0.253259
[64]	validation_0-error:0.037264	validation_0-gini:-0.253882
[65]	validation_0-error:0.037256	validation_0-gini:-0.254818
[66]	validation_0-error:0.037256	validation_0-gini:-0.255125
[67]	validation_0-error:0.037256	validation_0-gini:-0.255604
[68]	validation_0-error:0.037256	validation_0-gini:-0.256342
[69]	validation_0-error:0.037256	validation_0-gini:-0.25703
[70]	validation_0-error:0.037256	validation_0-gini:-0.257141
[71]	validation_0-error:0.037256	validation_0-gini:-0.257053
[72]	validation_0-error:0.037256	validation_0-gini:-0.257676
[73]	validation_0-error:0.037256	validation_0-gini:-0.258085
[74]	validation_0-error:0.037256	validation_0-gini:-0.258705
[75]	validation_0-error:0.037256	validation_0-gini:-0.258826
[76]	validation_0-error:0

[91]	validation_0-error:0.035844	validation_0-gini:-0.290224
[92]	validation_0-error:0.035844	validation_0-gini:-0.290416
[93]	validation_0-error:0.035844	validation_0-gini:-0.290443
[94]	validation_0-error:0.035844	validation_0-gini:-0.290594
[95]	validation_0-error:0.035844	validation_0-gini:-0.290378
[96]	validation_0-error:0.035844	validation_0-gini:-0.29062
[97]	validation_0-error:0.035844	validation_0-gini:-0.290693
[98]	validation_0-error:0.035844	validation_0-gini:-0.290922
[99]	validation_0-error:0.035844	validation_0-gini:-0.290963
  Best N trees =  100
  Best gini =  -0.290963
  Gini =  0.2909629068642212

Fold  4
[0]	validation_0-error:0.036626	validation_0-gini:-0.187675
Multiple eval metrics have been passed: 'validation_0-gini' will be used for early stopping.

Will train until validation_0-gini hasn't improved in 50 rounds.
[1]	validation_0-error:0.036626	validation_0-gini:-0.202724
[2]	validation_0-error:0.036626	validation_0-gini:-0.201479
[3]	validation_0-error:0.036

0.2760942227678944

In [13]:
# Save validation predictions for stacking/ensembling
val = pd.DataFrame()
val['id'] = id_train
val['target'] = y_valid_pred.values
val.to_csv('xgb_valid.csv', float_format='%.6f', index=False)

In [14]:
# Create submission file
sub = pd.DataFrame()
sub['id'] = id_test
sub['target'] = y_test_pred
sub.to_csv('xgb_submit.csv', float_format='%.6f', index=False)